{% include nav/toolkits/bathroom/menu.html %}

<div class="container">
    <div class="components">
        <!-- Section to manually enter teacher info -->
        <div class="div1">
            <h3>Enter Your Teacher’s Name</h3>
            <form id="teacherForm">
                <div class="form-group">
                    <label for="teacherFirstName">Teacher First Name:</label>
                    <input type="text" id="teacherFirstName" name="teacherFirstName" class="form-control" required>
                </div>
                <div class="form-group">
                    <label for="teacherLastName">Teacher Last Name:</label>
                    <input type="text" id="teacherLastName" name="teacherLastName" class="form-control" required>
                </div>
                <button type="button" class="btn btn-primary mt-2" onclick="saveTeacher()">Save Teacher</button>
            </form>
            <hr>
        </div>

        <!-- queue header and student queue info -->
        <div class="div2">
            <div class="queue-header">
                <h2 id="teacherQueueTitle">Teacher's Queue</h2>
            </div>
            <div class="queue-info">
                <p id="position">Your position in line: #</p>
                <p id="estimatedTime">Estimated time: ~ mins</p>
            </div>
            <hr>
        </div>
        <!-- students in line -->
        <div class="div3">
            <div class="queue-list">
                <p>Students currently in line:</p>
                <ul id="studentList">
                    <!-- Students will be listed here -->
                </ul>
                <div class="queue-buttons queue-actions">
                    <button type="button" onclick="window.addToQueue()">Add Me to Queue</button>
                    <button type="button" onclick="window.removeFromQueue()">Remove Me from Queue</button>
                    <button type="button" href="{{site.baseurl}}/bathroom/issues">Issues</button>
                </div>
            </div>
        </div>
    </div>
</div>

<!-- New "Request Pass" button -->
<div class="bottom-button-container">
    <button class="big-button" onclick="window.location.href='{{site.baseurl}}/bathroom/requestpass'">
        Request Pass
    </button>
</div>

<style>
    .bottom-button-container {
        display: flex;
        justify-content: center;
        margin-top: 20px;
    }

    .big-button {
        font-size: 40px;
        padding: 15px 30px;
        background-color: #28a745;
        color: white;
        border: none;
        border-radius: 10px;
        cursor: pointer;
    }

    .big-button:hover {
        background-color: #218838;
    }
</style>

<script type="module">
    import { javaURI, fetchOptions } from "{{site.baseurl}}/assets/js/api/config.js";

    const queueURI = javaURI + '/api/queue';
    const getUrl = queueURI + '/all';
    const addUrl = queueURI + '/add';
    const removeUrl = queueURI + '/remove';
    const approvalUrl = queueURI + '/approve';

    const postOptions = {
        ...fetchOptions,
        method: "POST",
    };
    const deleteOptions = {
        ...fetchOptions,
        method: "DELETE",
    };

    window.studentsInQueue = [];
    window.isApproving = false;

    // Retrieve stored teacher information
    const storedTeacherFirstName = localStorage.getItem("teacherFirstName");
    const storedTeacherLastName = localStorage.getItem("teacherLastName");

    if (storedTeacherFirstName && storedTeacherLastName) {
        document.getElementById("teacherQueueTitle").textContent = `${storedTeacherFirstName} ${storedTeacherLastName}'s Queue`;
    }

    // Save teacher input and refresh page
    window.saveTeacher = function () {
        const teacherFirstName = document.getElementById("teacherFirstName").value;
        const teacherLastName = document.getElementById("teacherLastName").value;

        if (teacherFirstName && teacherLastName) {
            localStorage.setItem("teacherFirstName", teacherFirstName);
            localStorage.setItem("teacherLastName", teacherLastName);

            alert("Teacher saved! Your queue will now update.");
            location.reload(); // Refresh to update UI
        } else {
            alert("Please enter both first and last name.");
        }
    };

    // Fetch queue data
    window.fetchQueueData = function () {
        fetch(getUrl, fetchOptions)
            .then(response => response.json())
            .then(data => {
                const queue = data.find(q => q.teacherFirstName === storedTeacherFirstName && q.teacherLastName === storedTeacherLastName);
                if (queue) {
                    window.studentsInQueue = queue.peopleQueue.split(',');
                    window.displayQueue();
                }
            })
            .catch(error => console.error("Error fetching data:", error));
    };

    // Display queue
    window.displayQueue = function () {
        const studentList = document.getElementById("studentList");
        studentList.innerHTML = "";

        window.studentsInQueue.forEach((student, index) => {
            const listItem = document.createElement("li");
            listItem.textContent = index === 0 ? `${student} (waiting for approval)` : student;
            if (student === window.studentName) listItem.style.color = "red";
            studentList.appendChild(listItem);
        });

        const position = window.studentsInQueue.indexOf(window.studentName) + 1;
        document.getElementById("position").textContent = `Your position in line: #${position}`;
        document.getElementById("estimatedTime").textContent = `Estimated time: ~${position * 3} mins`;
    };

    // Add student to queue
    window.addToQueue = function () {
        if (!storedTeacherFirstName || !storedTeacherLastName) {
            alert("Please enter your teacher's name first.");
            return;
        }

        const queuer = {
            teacherFirstName: storedTeacherFirstName,
            teacherLastName: storedTeacherLastName,
            studentName: window.studentName,
        };

        if (window.studentsInQueue.includes(window.studentName)) {
            alert("You are already in the queue.");
            return;
        }

        fetch(addUrl, {
            ...postOptions,
            body: JSON.stringify(queuer),
        }).then(response => {
            if (response.ok) window.fetchQueueData();
            else alert("Failed to add to queue.");
        }).catch(error => console.error("Error adding to queue:", error));
    };

    // Remove student from queue
    window.removeFromQueue = function () {
        const queuer = {
            teacherFirstName: storedTeacherFirstName,
            teacherLastName: storedTeacherLastName,
            studentName: window.studentName,
        };

        fetch(removeUrl, {
            ...deleteOptions,
            body: JSON.stringify(queuer),
        }).then(response => {
            if (response.ok) window.fetchQueueData();
            else alert("Failed to remove from queue.");
        }).catch(error => console.error("Error removing from queue:", error));
    };

    // Fetch queue data every 5 seconds
    setInterval(window.fetchQueueData, 5000);

    // Initial fetch
    window.fetchQueueData();
</script>

